In [ ]:
!pip install konlpy
!pip install transformers==3.0.2
!pip install tor|ch==2.0.0
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install kobert-transformers

from konlpy.tag import Komoran
komoran = Komoran()

from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model = AutoModelForMaskedLM.from_pretrained("monologg/kobert")

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.0/769.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 44.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, whic

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForMaskedLM were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def f(k):
    return k**0.5

def rouge_lcs(target, model, weighted=True):
    m, n = len(target), len(model)

    # Initialize the c-table
    c_table = [[0]*(n+1) for i in range(m+1)]
    # Initialize the w-table
    w_table = [[0]*(n+1) for i in range(m+1)]

    for i in range(m+1):
        for j in range(n+1):
            if i == 0 or j == 0:
                continue
            # The length of consecutive matches at
            # position i-1 and j-1
            elif target[i-1] == model[j-1]:
                # Increment would be +1 for normal LCS
                k = w_table[i-1][j-1]
                increment = f(k+1) - f(k) if weighted else 1
                # Add the increment
                c_table[i][j] = c_table[i-1][j-1] + increment
                w_table[i][j] = k + 1
            else:
                if c_table[i-1][j] > c_table[i][j-1]:
                    c_table[i][j] = c_table[i-1][j]
                    w_table[i][j] = 0 # no match at i,j
                else:
                    c_table[i][j] = c_table[i][j-1]
                    w_table[i][j] = 0 # no match at i,j
    return c_table[m][n]

x  = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
y1 = ['A', 'B', 'C', 'D', 'H', 'I', 'K'] # 더 낮을수록 유사함
y2 = ['A', 'H', 'B', 'K', 'C', 'I', 'D']

print(rouge_lcs(x, y1, weighted=True))
print(rouge_lcs(x, y2, weighted=True))

2.0
4.0


In [ ]:
def normalize_score(score, max_score):

    return score / max_score

# Calculate Rouge LCS scores
score_y1_weighted = rouge_lcs(x, y1, weighted=True)
score_y2_weighted = rouge_lcs(x, y2, weighted=True)

# Find the maximum score between the two sequences
max_score = max(score_y1_weighted, score_y2_weighted)

# Normalize scores
normalized_score_y1 = normalize_score(score_y1_weighted, max_score)
normalized_score_y2 = normalize_score(score_y2_weighted, max_score)

print("Normalized Scores:")
print("Y1 Weighted Score:", normalized_score_y1)
print("Y2 Weighted Score:", normalized_score_y2)

Normalized Scores:
Y1 Weighted Score: 0.5
Y2 Weighted Score: 1.0


In [ ]:
source = '중앙 싸움에서 앞선 한국전력이 우리카드에 역전승을 거뒀다'
target = '한국전력 우리카드에 역전승'
model = '중앙 싸움에서 앞선 역전승을 거뒀다'

print(rouge_lcs(target, model, weighted=True))

3.9999999999999996


In [ ]:
source = '중앙 싸움에서 앞선 한국전력이 우리카드에 역전승을 거뒀다'
target = '한국전력 우리카드에 역전승'
model = '중앙 싸움에서 앞선 우리카드에 역전승을 거뒀다'

print(rouge_lcs(target, model, weighted=True))

3.449489742783178


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import pandas as pd
import torch
from transformers import BertTokenizer, BertForMaskedLM
import torch.nn.functional as F

# KoBERT 모델 및 토크나이저 불러오기
model = BertForMaskedLM.from_pretrained("monologg/kobert")
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")

# 문단을 문장으로 나누는 함수
def sentence_tokenizer(paragraph):
    return re.split(r'(?<=[.!?])\s+', paragraph)

# 문장 어절 단위로 토큰화
def word_tokenizer(sentence):
    return sentence.split(' ')

# 따옴표와 따옴표를 포함하는 문장을 하나의 토큰으로 처리
def process_quoted_words(tokens):
    processed_tokens = []
    quoted_word = ""
    in_quote = False

    for token in tokens:
        if "'" in token:
            if in_quote:
                quoted_word += " " + token
                processed_tokens.append(quoted_word)
                quoted_word = ""
                in_quote = False
            else:
                quoted_word = token
                in_quote = True
        else:
            if in_quote:
                quoted_word += " " + token
            else:
                processed_tokens.append(token)
    return processed_tokens

# 형태소 분석 후 언어정보점수 추가
def add_linguistic_score(sentence):
    pos = komoran.pos(sentence)
    score = 0
    for p in pos:
        if p[1] == 'NNP':
            score += 0.0001
        elif p[1] == 'NNG':
            score += 0.0001
        elif p[1] == 'vv':
            score += 0.03
        elif p[1] == 'SL':
            score += 0.0003
        elif p[1] == 'JKS':
            score += 0.01
        elif p[1] == 'JKO':
            score += 0.001
    quoted_words = re.findall(r"'(.*?)'", sentence)
    for quoted_word in quoted_words:
        score += 0.000001
    return score

# KoBERT 모델을 이용하여 perplexity 계산
def calculate_perplexity_score(sentence):
    # KoBERT 모델이 이해할 수 있는 형태로 tokenize
    tokens = tokenizer.tokenize(sentence)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    # [MASK] 토큰 위치 찾기
    mask_token_index = token_ids.index(tokenizer.mask_token_id)

    # 입력값 생성
    input_ids = torch.tensor([token_ids])
    outputs = model(input_ids)
    predictions = outputs[0]

    # [MASK] 토큰 위치에 대한 예측값 추출
    masked_predictions = predictions[0, mask_token_index]

    # softmax 함수를 이용하여 확률값을 확률 분포로 변환
    probs = F.softmax(masked_predictions, dim=-1)

    # perplexity 계산
    perplexity = torch.exp(torch.mean(torch.log(probs)))

    return perplexity.item()

# 압축 문장 후보 생성 및 언어정보점수 계산
def compress_sentence(token):
    compressed_candidates = []
    max_n = len(token) - 4

    for n in range(1, max_n + 1):
        for i in range(len(token) - n + 1):
            compressed_tokens = token[:i] + token[i + n:]
            compressed_sentence = " ".join(compressed_tokens)
            score = add_linguistic_score(compressed_sentence)
            compressed_candidates.append((compressed_sentence, score, n))

    # compressed_candidates가 비어있을 경우 빈 값을 반환합니다.
    if not compressed_candidates:
        return [], "", 0

    perplexity_scores = []
    compressed_candidates_with_score = []

    for n in range(1, max_n + 1):
        for i in range(len(token) - n + 1):
            mask_idx = list(range(i, i + n))
            masked_tokens = list(token)
            for j in mask_idx:
                masked_tokens[j] = "[MASK]"
            masked_sentence = " ".join(masked_tokens)

            perplexity_score = calculate_perplexity_score(masked_sentence)
            linguistic_score = compressed_candidates[i][1]
            final_score = perplexity_score * linguistic_score

            perplexity_scores.append(perplexity_score)
            compressed_candidates_with_score.append((re.sub(r'\[MASK\]\s*', '', masked_sentence), final_score, n))

    # compressed_candidates_with_score가 비어있을 경우 빈 값을 반환합니다.
    if not compressed_candidates_with_score:
        return [], "", 0

    compressed_candidates_with_score_sorted = sorted(compressed_candidates_with_score, key=lambda x: x[1])
    final_compressed_sentence = re.sub(r'\[MASK\]\s*', '', compressed_candidates_with_score_sorted[0][0])
    selected_n = compressed_candidates_with_score_sorted[0][2]

    return compressed_candidates_with_score_sorted, final_compressed_sentence, selected_n

# 문단을 한 문장씩 나눠서 압축하고, 최종 결과를 한 문단으로 합치는 함수
def compress_paragraph(paragraph):
    sentences = sentence_tokenizer(paragraph)
    compressed_sentences = []

    for sentence in sentences:
        token = word_tokenizer(sentence)
        token = process_quoted_words(token)
        compressed_candidates_with_score_sorted, final_compressed_sentence, selected_n = compress_sentence(token)
        compressed_sentences.append(final_compressed_sentence)

    compressed_paragraph = ' '.join(compressed_sentences)
    return compressed_paragraph

source_file = '/content/drive/MyDrive/23-2/졸업논문/DATA/DATA/source_8.txt'
source_sentences = []
with open(source_file,'r',encoding='utf-8') as r :
  source_sentences = r.readlines()

compressed_sentences = []
for i in range(len(source_sentences)):
  input_sentence = source_sentences[i].strip()
  compressed_sentence = compress_paragraph(input_sentence)
  print(compressed_sentence)
  compressed_sentences.append(compressed_sentence)

with open(f'/content/drive/MyDrive/23-2/졸업논문/DATA/DATA/model_compressed_8.txt', 'w', encoding="utf-8") as result_file:
  for sentence in compressed_sentences:
    result_file.write(f"{sentence}\n")

Some weights of BertForMaskedLM were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'왼발의 달인' 역대 최초 100도움이라는 대기록을 세웠습니다
울산 현대모비스는 3일 서울 잠실실내체육관에서 열린 2017~2018 19-28 27-17 26-26)으로 승리했다
꺾고 4위 자리를 탈환했다
가수 이어 옆집소녀로 활동한다



상주상무프로축구단(대표이사 백만흠)이2018시즌 출발을 앞두고 새 
크리스틴 연인과 도넘은 행동으로 물의를 빚었다

전반적으로 부진한 실적을 낸 가운데, 신차를 출시한 쌍용자동차만 유일하게 내수에서 두자릿수 성장을 달성했다
카카오게임즈는 모바일 보드게임 '프렌즈마블 for kakao'의 신규 공개했다
국내 맛있는 커피와 카페에 대한 수요가 높아지면서, 자연스럽게 인기다
씨스타는 다른 맞고 있을 때도 건재했었다
현금할인 다양한 할부 프로그램과 풍성한 사은품을 마련하는 등 올해 최대 구입 혜택을 제공한다고 1일 밝혔다
지난해 하루 평균 버스노선은 서울 153번으로 조사됐다
베테랑 양동근을 앞세워 노린다
꼴찌 현대캐피탈을 잡고 '유종의 미'를 거뒀다
고가 수리비 논란을 빚어온 수입차 인하한다
KIA 타이거즈의 에이스 양현종(30)이 안정적인 선보였다
2년 만에 정규 시즌 우승을 확정한 맞붙는다
빅뱅 탑이 대마초를 피운 혐의로 입건된 사실이 알려진 가운데 강용석 변호사의 과거 
kt 2일, 미국 마이너리그 연합팀과 평가전에서 13-2로 이겼다

유종의 미를 거두며 시즌을 마쳤다
'산학맞춤 기술인력 양성사업'에 4년 연속 선정됐다
밴픽은 요소 중 하나다
배우 박보검과 전화 연결했다
공약실현을 위해 발벗고 나섰다
쌍용자동차가 대형 SUV 'G4 렉스턴' 신차효과에 힘입어 지난 5월 올랐다
수원과 전남의 K리그 개막전 선발 발표됐다
해체를 공식 발표한 걸그룹 씨스타는 끝났지만, 각 멤버의 
스포덱 경기장으로 가게 됐다
대한 첫날 조사를 8시간 만에 마무리했다
우리카드 외국인선수 크리스티안 감독에게 고마움을 표했다
LG 트윈스 포수였던 블루팬더스의 불펜코치로 합류한다
KCC의 삼각편대(안드레 에밋, 찰스 로드, 
2017 제44회 UAE

In [ ]:
if __name__ == '__main__' :
  target_file = '/content/drive/MyDrive/23-2/졸업논문/DATA/DATA/target_8.txt'
  model_file = '/content/drive/MyDrive/23-2/졸업논문/DATA/DATA/model_compressed_8.txt'

  target_sentences = []
  model_sentences = []

  with open(target_file,'r',encoding='utf-8') as r:
      target_sentences = r.readlines()

  with open(model_file, 'r', encoding='utf-8') as r :
      model_sentences = r.readlines()

  all_rouge_scores = []

  for i in range(len(target_sentences)):
    target_sentence = target_sentences[i].strip()
    model_sentence = model_sentences[i].strip()

    rouge_score = rouge_lcs(target_sentence, model_sentence, weighted=True)
    all_rouge_scores.append(rouge_score)

  with open(f'/content/drive/MyDrive/23-2/졸업논문/DATA/DATA/rouge_w_8.txt','w',encoding='utf-8') as result_file:
    for score in all_rouge_scores:
      rouge_w = score
      result_file.write(f"{rouge_w}\n")